Based on Michael Eisen's [Neural Network and Deep Learning Book](http://neuralnetworksanddeeplearning.com/chap1.html).

In [39]:
"""
Implement the stochastic gradient descent learning algorithm
Gradients are calculated using backpropagation
Updated from Neural Network and Deep Learning Book to work for Python 3 

"""

import random
import numpy as np

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))


class Network(object):

    def __init__(self, sizes):
        """
        sizes - List of number of neurons in each of the layers
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        # Initialize bias and weights randomly with Gaussian(0,1)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in list(zip(sizes[:-1], sizes[1:]))]

        
    def feedforward(self, a):
        """
        a - Activation
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        Train using mini-batch stochastic gradient descent.
        training_data - Tuple of training inputs, desired outputs
        epochs - Number of times to repeat
        mini_batch_size - Number of inputs to use
        eta - Learning rate
        test_data - Useful for tracking progress. Optional
        """
        if test_data: n_test = len(test_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
                

    def initialize_gradients(self):
        """
        Initialize gradients of weights and biases with zeros
        """
        gradient_b = [np.zeros(b.shape) for b in self.biases]
        gradient_w = [np.zeros(w.shape) for w in self.weights]
        
        return gradient_b, gradient_w
    
        
    def update_mini_batch(self, mini_batch, eta):
        """
        Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        mini_batch - A list of tuples (x, y)
        eta - Learning rate
        """
        gradient_b, gradient_w = self.initialize_gradients()
        
        for x, y in mini_batch:
            delta_gradient_b, delta_gradient_w = self.backprop(x, y)
            gradient_b = [nb + dnb for nb, dnb in list(zip(gradient_b, delta_gradient_b))]
            gradient_w = [nw + dnw for nw, dnw in list(zip(gradient_w, delta_gradient_w))]
        
        self.weights = [w - (eta/len(mini_batch)) * nw
                        for w, nw in list(zip(self.weights, gradient_w))]
        
        self.biases = [ b - (eta/len(mini_batch)) * nb
                       for b, nb in list(zip(self.biases, gradient_b))]

        
    def backprop(self, x, y):
        """
        Compute the gradient for the cost function C_x
        """
        gradient_b, gradient_w = self.initialize_gradients()
        
        # Feedforward
        activation = x
        # List to store all the activations, layer by layer
        activations = [x] 
        # List to store all the z vectors, layer by layer
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        gradient_b[-1] = delta
        gradient_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # l = 1 (last layer of neurons), l = 2 (second-last)
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            gradient_b[-l] = delta
            gradient_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (gradient_b, gradient_w)

    
    def evaluate(self, test_data):
        """
        Compute the number of test inputs with the correct outputs.
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    
    def cost_derivative(self, output_activations, y):
        """
        output_activations - Activations of the outputs
        y - Output
        """
        return (output_activations - y)



In [40]:
"""
Implement a wrapper to load the MNIST image data
"""

import _pickle as cPickle
import gzip
import numpy as np

def load_data():
    """
    Return the MNIST data as:
    training - 50,000, 28*28=784 px/image, digit values (0...9)
    validation - 10,000 additional images
    test - 10,000 additional images
    """

    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='iso-8859-1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data_wrapper():
    """
    Return the tuple (training_data, validation_data, test_data)
    training_data - A list of 50,000 (x, y) tuples  
    x - 784-dim ndarray containing the input image
    y - A 10-dim ndarray of unit vector for correct digit for x
    validation_data - A list of 10,000 (x, y) additional tuples
    test_data - A list of 10,000 (x, y) additional tuples
    """
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """
    Return a 10-dim unit vector for digits(0...9), one-hot encoded
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [41]:
training_data, validation_data, test_data = load_data_wrapper()

In [42]:
net = Network([784, 30, 10])
# 30 epochs, 10 neurons, learning rate = 3
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8171 / 10000
Epoch 1: 8327 / 10000
Epoch 2: 8372 / 10000
Epoch 3: 8418 / 10000
Epoch 4: 8471 / 10000
Epoch 5: 8485 / 10000
Epoch 6: 8477 / 10000
Epoch 7: 8477 / 10000
Epoch 8: 8513 / 10000
Epoch 9: 8507 / 10000
Epoch 10: 8521 / 10000
Epoch 11: 8524 / 10000
Epoch 12: 8523 / 10000
Epoch 13: 8538 / 10000
Epoch 14: 8527 / 10000
Epoch 15: 8547 / 10000
Epoch 16: 8537 / 10000
Epoch 17: 8533 / 10000
Epoch 18: 8544 / 10000
Epoch 19: 8546 / 10000
Epoch 20: 8556 / 10000
Epoch 21: 8531 / 10000
Epoch 22: 8538 / 10000
Epoch 23: 8564 / 10000
Epoch 24: 8540 / 10000
Epoch 25: 8553 / 10000
Epoch 26: 8531 / 10000
Epoch 27: 8558 / 10000
Epoch 28: 8540 / 10000
Epoch 29: 8552 / 10000


In [43]:
net = Network([784, 100, 10])
# 30 epochs, 100 neurons, learning rate = 3
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 6607 / 10000
Epoch 1: 6709 / 10000
Epoch 2: 7503 / 10000
Epoch 3: 7617 / 10000
Epoch 4: 7676 / 10000
Epoch 5: 8571 / 10000
Epoch 6: 8571 / 10000
Epoch 7: 8640 / 10000
Epoch 8: 8624 / 10000
Epoch 9: 8656 / 10000
Epoch 10: 8670 / 10000
Epoch 11: 8660 / 10000
Epoch 12: 8686 / 10000
Epoch 13: 8689 / 10000
Epoch 14: 8671 / 10000
Epoch 15: 8692 / 10000
Epoch 16: 8680 / 10000
Epoch 17: 8708 / 10000
Epoch 18: 8693 / 10000
Epoch 19: 8699 / 10000
Epoch 20: 8705 / 10000
Epoch 21: 8693 / 10000
Epoch 22: 8707 / 10000
Epoch 23: 8700 / 10000
Epoch 24: 8692 / 10000
Epoch 25: 8703 / 10000
Epoch 26: 8708 / 10000
Epoch 27: 8703 / 10000
Epoch 28: 8699 / 10000
Epoch 29: 8700 / 10000
